In [2]:
!python --version
!which python

Python 3.12.4
/Users/wakala/venvs/versions/3.12.4/GeneRegulation/bin/python


In [85]:
import os
import warnings
import pandas as pd
import scanpy as sc
import pyscenic as ps

from dask import delayed
from dask.dataframe import from_delayed
from dask.dataframe.dispatch import make_meta
from arboreto.core import create_graph, to_tf_matrix, target_gene_indices, infer_partial_network
from pyscenic.prune import _prepare_client
from arboreto.algo import genie3, grnboost2, _prepare_client, _prepare_input
from pyscenic.cli.pyscenic import load_exp_matrix, find_adjacencies_command, load_tf_names

warnings.filterwarnings('ignore')
print("Scanpy version: ", sc.__version__)
print("PySCENIC version: ", ps.__version__)

Scanpy version:  1.10.4
PySCENIC version:  0.12.1+8.gd2309fe


In [4]:
adata = sc.read_10x_h5('/Users/wakala/Desktop/Works/Data/singlecell/filtered_feature_bc_matrix.h5')
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 10194 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [7]:
df = adata.to_df()
df.shape

(10194, 36601)

In [9]:
df.to_csv('~/Desktop/Works/Data/singlecell/matrix.tsv', sep='\t', index=True)

In [24]:
df = load_exp_matrix('~/Desktop/Works/Data/singlecell/matrix.tsv')
df.head()

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1,...,AC133551.1,AC136612.1,AC136616.1,AC136616.3,AC136616.2,AC141272.1,AC023491.2,AC007325.1,AC007325.4,AC007325.2
AAACCCACATAACTCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
AAACCCACATGTAACC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCCAGTGAGTCAG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCCAGTGCTTATG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAACAGTCAGTT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
adata.var_names.to_frame().iloc[1000:1005]

,0
MAST2,MAST2
PIK3R3,PIK3R3
AL358075.1,AL358075.1
AL358075.2,AL358075.2
TSPAN1,TSPAN1


In [35]:
tf_names = load_tf_names('/Users/wakala/Desktop/Works/Data/singlecell/allTFs_hg38.txt')

In [36]:
n_total_genes = len(df.columns)
n_matching_genes = len(df.columns.isin(tf_names))
n_total_genes, n_matching_genes

(36601, 36601)

In [38]:
float(n_matching_genes) / n_total_genes

1.0

In [59]:
client, shutdown_callback = _prepare_client('local')
client, shutdown_callback


(<Client: 'tcp://127.0.0.1:62523' processes=5 threads=10, memory=16.00 GiB>,
 <function arboreto.algo._prepare_client.<locals>.close_client_and_local_cluster(verbose=False)>)

In [44]:
method = genie3
network = method(expression_data=df, tf_names=tf_names, verbose=True, client_or_address=client,seed=999)

preparing dask client
parsing input
creating dask graph
not shutting down client, client was created externally
finished


TypeError: Must supply at least one delayed object

In [45]:
demo = grnboost2(expression_data=df)

2025-01-02 19:33:15,369 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:59090 (pid=89553) exceeded 95% memory budget. Restarting...
2025-01-02 19:33:15,430 - distributed.scheduler - ERROR - Removing worker 'tcp://127.0.0.1:59090' caused the cluster to lose scattered data, which can't be recovered: {'ndarray-9162eb67e6564205d4f5a7e624fd6502'} (stimulus_id='handle-worker-cleanup-1735817595.430141')
2025-01-02 19:33:15,432 - distributed.nanny - WARNING - Restarting worker
2025-01-02 19:33:15,431 - distributed.worker - ERROR - Could not find data: ['ndarray-9162eb67e6564205d4f5a7e624fd6502']
2025-01-02 19:33:15,431 - distributed.worker - ERROR - Could not find data: ['ndarray-9162eb67e6564205d4f5a7e624fd6502']
2025-01-02 19:33:15,437 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:59088 failed to acquire keys: ('ndarray-9162eb67e6564205d4f5a7e624fd6502',)
2025-01-02 19:33:15,438 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:59086 failed to acquire keys

TypeError: Must supply at least one delayed object

In [48]:
expression_matrix, gene_names, tf_names = _prepare_input(df, df.index.to_list(), tf_names)

In [55]:
graph = create_graph(expression_matrix, gene_names, tf_names, client=client, regressor_type='GBM', regressor_kwargs={
    'learning_rate': 0.01,
    'n_estimators': 5000,  # can be arbitrarily large
    'max_features': 0.1,
    'subsample': 0.9
})

2025-01-02 19:43:25,267 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:61167 -> tcp://127.0.0.1:61168
Traceback (most recent call last):
  File "/Users/wakala/venvs/versions/3.12.4/GeneRegulation/lib/python3.12/site-packages/tornado/iostream.py", line 962, in _handle_write
    num_bytes = self.write_to_fd(self._write_buffer.peek(size))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wakala/venvs/versions/3.12.4/GeneRegulation/lib/python3.12/site-packages/tornado/iostream.py", line 1124, in write_to_fd
    return self.socket.send(data)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 55] No buffer space available

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/wakala/venvs/versions/3.12.4/GeneRegulation/lib/python3.12/site-packages/distributed/worker.py", line 1797, in get_data
    response = await comm.read(deserializers=serializers)
     

2025-01-02 19:43:25,275 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:61168 failed to acquire keys: ('ndarray-6807fcce217331d5e197828e9a0ef9c0',)


TypeError: Must supply at least one delayed object

In [78]:
tf_matrix, tf_matrix_gene_names = to_tf_matrix(df.T.values, df.index.to_list(), tf_names)

In [79]:
client, shutdown_callback = _prepare_client('local')
client, shutdown_callback
future_tf_matrix = client.scatter(tf_matrix, broadcast=True)

In [80]:
[future_tf_matrix_gene_names] = client.scatter([tf_matrix_gene_names], broadcast=True)

In [81]:
delayed_link_dfs = []  # collection of delayed link DataFrames
delayed_meta_dfs = []

for target_gene_index in target_gene_indices(gene_names, 'all'):
    target_gene_name = delayed(gene_names[target_gene_index], pure=True)
    target_gene_expression = delayed(expression_matrix[:, target_gene_index], pure=True)

    delayed_link_df = delayed(infer_partial_network, pure=True)(
                'GBM',
                {'learning_rate': 0.01, 'n_estimators': 5000, 'max_features': 0.1, 'subsample': 0.9},
                future_tf_matrix, 
                future_tf_matrix_gene_names,
                target_gene_name, 
                target_gene_expression, 
                False, 
                25, 
                666
            )
    
    if delayed_link_df is not None:
        delayed_link_dfs.append(delayed_link_df)


In [93]:
len(delayed_link_dfs)

36601

In [87]:
_GRN_SCHEMA = make_meta({'TF': str, 'target': str, 'importance': float})
_META_SCHEMA = make_meta({'target': str, 'n_estimators': int})


In [95]:
_META_SCHEMA

,target,n_estimators


In [91]:

all_links_df = from_delayed(dfs=delayed_link_dfs, meta=_GRN_SCHEMA)
# all_meta_df = from_delayed(dfs=delayed_meta_dfs, meta=_META_SCHEMA)